# Соревнования про медицину

https://inclass.kaggle.com/c/competition-2-yandex-shad-spring-2017

В файлах X.train и y.train задана обучающая выборка, X.test - выборка, для которой необходимо сделать предсказание.
Признаки

Для каждого пациента задано описание в виде 1330 признаков:

    Столбцы с V2 по V331 соответствуют различным медицинским показателям.
    Столбцы с V332 по V1331 соответствуют генетическими данными пациента.

Обратите внимание, что в случае неизвестного значения указано NaN.
Типы признаков

В файле MetaData содержится подробное описание типов каждого столбца признаков:

    Numeric - числовые признаки;
    Category - порядок значений нам не важен;
    Ordered Category - порядок значений важен;
    Генетические данные - их значения соответствуют частоте конкретного аллеля.



In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For sklearn 0.18
# import sklearn.model_selection as skcv
import sklearn.cross_validation as skcv
import sklearn.ensemble as sk
import sklearn.preprocessing as skpp
import sklearn.metrics as skm

In [43]:
meta_data = pd.read_csv('cont_02/MetaData.csv')
X_train = pd.read_csv('cont_02/X.train.csv')
X_test = pd.read_csv('cont_02/X.test.csv')
Y_train = pd.read_csv('cont_02/y.train.csv')

In [84]:
print('X_train.shape=', X_train.shape)
Y_train.head(15)

X_train.shape= (4099, 1330)


,label
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


## Базовое решение

In [45]:
imputer = skpp.Imputer(strategy='most_frequent')
X_train_mean = imputer.fit_transform(X_train.values)
X_test_mean = imputer.transform(X_test.values)

In [90]:
clf = sk.RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=1)
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [91]:
np.mean(skcv.cross_val_score(clf, X_train_mean, Y_train.label.values, scoring='roc_auc', cv=5))

0.81565107584450658

## Идеи Алексея
### Заменить NaN на нули, снабдить колонки копиями с isNaN-признаком (0/1)

In [89]:
def denan(X):

    X2 = X.copy()
    X2[pd.isnull(X)] = 1
    X2[~(pd.isnull(X))] = 0

    X2.columns = [x + 'N' for x in X2.columns]
    X_nan = pd.concat([X, X2], axis=1)
    X_nan[pd.isnull(X_nan)] = 0
    return X_nan

X_train_nan = denan(X_train)

In [92]:
clf = sk.RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=1)
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [93]:
np.mean(skcv.cross_val_score(clf, X_train_nan, Y_train.label.values, scoring='roc_auc', cv=5))

0.82582355230844562

### Заменить NaN на нули, снабдить колонки копиями с isNaN-признаком (min-1/max+1)